In [1]:
%load_ext autoreload
%autoreload 2

In [89]:
from metal.mmtl.trainer import MultitaskTrainer
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.slicing.slice_model import SliceModel
from metal.mmtl.slicing.payloads import remap_labelsets

In [3]:
SEED = 1

### Initialize normal payloads

In [4]:
task_kwargs = {
    "dl_kwargs": {"batch_size": 16},
    "freeze_bert":False,
    "bert_model": 'bert-large-uncased',
    "max_len": 128,
    "attention": False,
    "dropout": 0.1,
}
task_names = ["RTE"]

In [5]:
%%time

tasks, payloads = create_glue_tasks_payloads(task_names, **task_kwargs)

Using random seed: 136522
Loading RTE Dataset



CPU times: user 26.1 s, sys: 2.66 s, total: 28.8 s
Wall time: 29.7 s


In [6]:
tasks, payloads

([ClassificationTask(name=RTE, loss_multiplier=1.00)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=valid),
  Payload(RTE_test: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=test)])

### Load existing baseline model

In [7]:
model = MetalModel(tasks, seed=SEED, verbose=False)

In [8]:
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads
from metal.mmtl.metal_model import MetalModel

SEED = 321
baseline = MetalModel(tasks, seed=SEED, verbose=False)

import os
import torch
model_dir = '/dfs/scratch0/vschen/metal-mmtl/logs/2019_04_27/16_28_12'
model_path = os.path.join(model_dir, 'best_model.pth')
baseline.load_weights(model_path)

#### Valdiate baseline performance

In [9]:
baseline.score(payloads[0])

{'RTE/RTE_train/RTE_gold/accuracy': 0.9714859437751004}

In [10]:
baseline.score(payloads[1])

{'RTE/RTE_valid/RTE_gold/accuracy': 0.8267148014440433}

### Visualize some slices

In [133]:
# from metal.mmtl.glue.glue_slices import *

In [134]:
# import spacy
# nlp = spacy.load("en_core_web_sm")

# import random
# ds = payloads[2].data_loader.dataset
# rand_idx = list(range(len(sentences)))
# random.shuffle(rand_idx)

# counter = 0
# for idx in rand_idx:
#     sentence = ds.sentences[idx]
#     in_slice = has_preposition_s2(ds, idx)
#     if in_slice:
#         counter += 1
#         print(sentence, "->", in_slice)
# #         import pdb; pdb.set_trace()
                
# print(counter)

### Initialize slice payloads

In [138]:
rte_slices = [
    "has_preposition_s2", 
    "common_negation",
    "has_numerical_date",
    "more_people",
    "dash_semicolon"
]

In [135]:
# Create tasks and payloads
task_kwargs.update({"slice_dict": {"RTE": rte_slices}})
task_kwargs['attention'] = None

tasks_slice, payloads_slice = create_glue_tasks_payloads(
    task_names, **task_kwargs
)

Using random seed: 632688
Loading RTE Dataset



Added label_set with 2490/2490 labels for task RTE_slice:has_preposition_s2:ind to payload RTE_train.
Added label_set with 1836/2490 labels for task RTE_slice:has_preposition_s2:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:common_negation:ind to payload RTE_train.
Added label_set with 355/2490 labels for task RTE_slice:common_negation:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:has_numerical_date:ind to payload RTE_train.
Added label_set with 83/2490 labels for task RTE_slice:has_numerical_date:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:more_people:ind to payload RTE_train.
Added label_set with 64/2490 labels for task RTE_slice:more_people:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:dash_semicolon:ind to payload RTE_train.
Added label_set with 985/2490 labels for task RTE_slice:dash_semicolon:pred to payload RTE_train.
Added la

### Evaluate baseline slices

In [139]:
tasks_slice, payloads_slice

([ClassificationTask(name=RTE, loss_multiplier=1.00),
  ClassificationTask(name=RTE_slice:has_preposition_s2:ind, loss_multiplier=0.10),
  ClassificationTask(name=RTE_slice:has_preposition_s2:pred, loss_multiplier=0.10),
  ClassificationTask(name=RTE_slice:common_negation:ind, loss_multiplier=0.10),
  ClassificationTask(name=RTE_slice:common_negation:pred, loss_multiplier=0.10),
  ClassificationTask(name=RTE_slice:has_numerical_date:ind, loss_multiplier=0.10),
  ClassificationTask(name=RTE_slice:has_numerical_date:pred, loss_multiplier=0.10),
  ClassificationTask(name=RTE_slice:more_people:ind, loss_multiplier=0.10),
  ClassificationTask(name=RTE_slice:more_people:pred, loss_multiplier=0.10),
  ClassificationTask(name=RTE_slice:dash_semicolon:ind, loss_multiplier=0.10),
  ClassificationTask(name=RTE_slice:dash_semicolon:pred, loss_multiplier=0.10)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:has_preposition_s2:ind': 'RTE_slice:has_preposition_s2:ind', 'RTE_sli

In [140]:
import copy
eval_payload = copy.deepcopy(payloads_slice[1])

# NOTE: we need to retarget slices to the original RTE head
for slice_name in rte_slices:
    label_name = f"RTE_slice:{slice_name}"
    for slice_head_type in ['ind', 'pred']:
        eval_payload._retarget_labelset(f'{label_name}:{slice_head_type}', 'RTE')

labelset 'RTE_slice:has_preposition_s2:ind' -> task 'RTE' (originally, RTE_slice:has_preposition_s2:ind).
labelset 'RTE_slice:has_preposition_s2:pred' -> task 'RTE' (originally, RTE_slice:has_preposition_s2:pred).
labelset 'RTE_slice:common_negation:ind' -> task 'RTE' (originally, RTE_slice:common_negation:ind).
labelset 'RTE_slice:common_negation:pred' -> task 'RTE' (originally, RTE_slice:common_negation:pred).
labelset 'RTE_slice:has_numerical_date:ind' -> task 'RTE' (originally, RTE_slice:has_numerical_date:ind).
labelset 'RTE_slice:has_numerical_date:pred' -> task 'RTE' (originally, RTE_slice:has_numerical_date:pred).
labelset 'RTE_slice:more_people:ind' -> task 'RTE' (originally, RTE_slice:more_people:ind).
labelset 'RTE_slice:more_people:pred' -> task 'RTE' (originally, RTE_slice:more_people:pred).
labelset 'RTE_slice:dash_semicolon:ind' -> task 'RTE' (originally, RTE_slice:dash_semicolon:ind).
labelset 'RTE_slice:dash_semicolon:pred' -> task 'RTE' (originally, RTE_slice:dash_sem

In [141]:
eval_payload

Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:has_preposition_s2:ind': 'RTE', 'RTE_slice:has_preposition_s2:pred': 'RTE', 'RTE_slice:common_negation:ind': 'RTE', 'RTE_slice:common_negation:pred': 'RTE', 'RTE_slice:has_numerical_date:ind': 'RTE', 'RTE_slice:has_numerical_date:pred': 'RTE', 'RTE_slice:more_people:ind': 'RTE', 'RTE_slice:more_people:pred': 'RTE', 'RTE_slice:dash_semicolon:ind': 'RTE', 'RTE_slice:dash_semicolon:pred': 'RTE'}], split=valid)

In [142]:
baseline.score(eval_payload)

{'RTE/RTE_valid/RTE_gold/accuracy': 0.8267148014440433,
 'RTE/RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy': 0.5306859205776173,
 'RTE/RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy': 0.8235294117647058,
 'RTE/RTE_valid/RTE_slice:common_negation:ind/accuracy': 0.37545126353790614,
 'RTE/RTE_valid/RTE_slice:common_negation:pred/accuracy': 0.9090909090909091,
 'RTE/RTE_valid/RTE_slice:has_numerical_date:ind/accuracy': 0.3971119133574007,
 'RTE/RTE_valid/RTE_slice:has_numerical_date:pred/accuracy': 1.0,
 'RTE/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.3862815884476534,
 'RTE/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.6666666666666666,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.4584837545126354,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.8333333333333334}

### Initialize and train slice model

In [143]:
from metal.mmtl.slicing.tasks import convert_to_slicing_tasks
slicing_tasks = convert_to_slicing_tasks(tasks_slice)
slicing_tasks

Modifying RTE out_features from 2 -> 1
Modifying RTE_slice:has_preposition_s2:ind out_features from 2 -> 1
Modifying RTE_slice:has_preposition_s2:pred out_features from 2 -> 1
Modifying RTE_slice:common_negation:ind out_features from 2 -> 1
Modifying RTE_slice:common_negation:pred out_features from 2 -> 1
Modifying RTE_slice:has_numerical_date:ind out_features from 2 -> 1
Modifying RTE_slice:has_numerical_date:pred out_features from 2 -> 1
Modifying RTE_slice:more_people:ind out_features from 2 -> 1
Modifying RTE_slice:more_people:pred out_features from 2 -> 1
Modifying RTE_slice:dash_semicolon:ind out_features from 2 -> 1
Modifying RTE_slice:dash_semicolon:pred out_features from 2 -> 1


[BinaryClassificationTask(name=RTE, loss_multiplier=1.00, slice_head_type=None),
 BinaryClassificationTask(name=RTE_slice:has_preposition_s2:ind, loss_multiplier=0.10, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:has_preposition_s2:pred, loss_multiplier=0.10, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:common_negation:ind, loss_multiplier=0.10, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:common_negation:pred, loss_multiplier=0.10, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:has_numerical_date:ind, loss_multiplier=0.10, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:has_numerical_date:pred, loss_multiplier=0.10, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:more_people:ind, loss_multiplier=0.10, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:more_people:pred, loss_multiplier=0.10, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:dash_semicolon:ind

In [145]:
# slice_model = SliceModel(slicing_tasks, seed=SEED, verbose=False)
slice_model = MetalModel(slicing_tasks, seed=SEED, verbose=False)

In [146]:
# load weights from the base model
base_state_dict = baseline.state_dict()
slice_state_dict = slice_model.state_dict()
del base_state_dict['head_modules.RTE.module.module.weight']
del base_state_dict['head_modules.RTE.module.module.bias']
slice_state_dict.update(base_state_dict)
slice_model.load_state_dict(slice_state_dict)

In [147]:
%%time
trainer = MultitaskTrainer(seed=SEED)
trainer.train_model(
    slice_model,
    payloads_slice,
    checkpoint_metric="RTE/RTE_valid/RTE_gold/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=1e-5,
#     l2=1e-3,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=50,
    progress_bar=True,
    checkpoint_best=True,
    checkpoint_cleanup=True
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 156 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/vschen/metal-mmtl/logs/2019_04_29/01_08_19/config.json


[0.10 epo]: RTE:[RTE_train/RTE_gold/loss=7.63e-01, RTE_valid/RTE_gold/accuracy=6.50e-01] RTE_slice:has_preposition_s2:ind:[RTE_train/RTE_slice:has_preposition_s2:ind/loss=7.06e-02, RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy=3.90e-01] RTE_slice:has_preposition_s2:pred:[RTE_train/RTE_slice:has_preposition_s2:pred/loss=6.19e-02, RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy=6.72e-01] RTE_slice:common_negation:ind:[RTE_train/RTE_slice:common_negation:ind/loss=6.63e-02, RTE_valid/RTE_slice:common_negation:ind/accuracy=7.29e-01] RTE_slice:common_negation:pred:[RTE_train/RTE_slice:common_negation:pred/loss=7.87e-02, RTE_valid/RTE_slice:common_negation:pred/accuracy=5.15e-01] RTE_slice:has_numerical_date:ind:[RTE_train/RTE_slice:has_numerical_date:ind/loss=6.95e-02, RTE_valid/RTE_slice:has_numerical_date:ind/accuracy=7.26e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=7.85e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=1.12e-01] RTE_slice:more_peopl

[0.62 epo]: RTE:[RTE_train/RTE_gold/loss=2.82e-01, RTE_valid/RTE_gold/accuracy=8.12e-01] RTE_slice:has_preposition_s2:ind:[RTE_train/RTE_slice:has_preposition_s2:ind/loss=6.61e-02, RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy=6.90e-01] RTE_slice:has_preposition_s2:pred:[RTE_train/RTE_slice:has_preposition_s2:pred/loss=5.42e-02, RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy=7.25e-01] RTE_slice:common_negation:ind:[RTE_train/RTE_slice:common_negation:ind/loss=5.74e-02, RTE_valid/RTE_slice:common_negation:ind/accuracy=7.15e-01] RTE_slice:common_negation:pred:[RTE_train/RTE_slice:common_negation:pred/loss=7.39e-02, RTE_valid/RTE_slice:common_negation:pred/accuracy=5.76e-01] RTE_slice:has_numerical_date:ind:[RTE_train/RTE_slice:has_numerical_date:ind/loss=5.57e-02, RTE_valid/RTE_slice:has_numerical_date:ind/accuracy=8.99e-01] RTE_slice:has_numerical_date:pred:[RTE_train/RTE_slice:has_numerical_date:pred/loss=5.52e-02, RTE_valid/RTE_slice:has_numerical_date:pred/accuracy=1.00e+

[1.03 epo]: RTE:[RTE_train/RTE_gold/loss=2.02e-01, RTE_valid/RTE_gold/accuracy=8.05e-01] RTE_slice:has_preposition_s2:ind:[RTE_train/RTE_slice:has_preposition_s2:ind/loss=6.35e-02, RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy=7.33e-01] RTE_slice:has_preposition_s2:pred:[RTE_train/RTE_slice:has_preposition_s2:pred/loss=4.63e-02, RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy=7.25e-01] RTE_slice:common_negation:ind:[RTE_train/RTE_slice:common_negation:ind/loss=5.45e-02, RTE_valid/RTE_slice:common_negation:ind/accuracy=8.16e-01] RTE_slice:common_negation:pred:[RTE_train/RTE_slice:common_negation:pred/loss=6.41e-02, RTE_valid/RTE_slice:common_negation:pred/accuracy=9.09e-01] RTE_slice:has_numerical_date:ind:[RTE_train/RTE_slice:has_numerical_date:ind/loss=4.48e-02, RTE_valid/RTE_slice:has_numerical_date:ind/accuracy=9.57e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=4.94e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.42e-01] RTE_slice:more_peopl

[1.54 epo]: RTE:[RTE_train/RTE_gold/loss=1.83e-01, RTE_valid/RTE_gold/accuracy=8.19e-01] RTE_slice:has_preposition_s2:ind:[RTE_train/RTE_slice:has_preposition_s2:ind/loss=6.19e-02, RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy=7.36e-01] RTE_slice:has_preposition_s2:pred:[RTE_train/RTE_slice:has_preposition_s2:pred/loss=4.10e-02, RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy=7.99e-01] RTE_slice:common_negation:ind:[RTE_train/RTE_slice:common_negation:ind/loss=4.81e-02, RTE_valid/RTE_slice:common_negation:ind/accuracy=8.77e-01] RTE_slice:common_negation:pred:[RTE_train/RTE_slice:common_negation:pred/loss=4.69e-02, RTE_valid/RTE_slice:common_negation:pred/accuracy=9.09e-01] RTE_slice:has_numerical_date:ind:[RTE_train/RTE_slice:has_numerical_date:ind/loss=3.82e-02, RTE_valid/RTE_slice:has_numerical_date:ind/accuracy=9.75e-01] RTE_slice:has_numerical_date:pred:[RTE_train/RTE_slice:has_numerical_date:pred/loss=5.72e-02, RTE_valid/RTE_slice:has_numerical_date:pred/accuracy=1.00e+

...

[49.03 epo]: RTE:[RTE_train/RTE_gold/loss=1.05e-01, RTE_valid/RTE_gold/accuracy=7.36e-01] RTE_slice:has_preposition_s2:ind:[RTE_train/RTE_slice:has_preposition_s2:ind/loss=6.48e-03, RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy=9.78e-01] RTE_slice:has_preposition_s2:pred:[RTE_train/RTE_slice:has_preposition_s2:pred/loss=1.37e-02, RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy=7.40e-01] RTE_slice:common_negation:ind:[RTE_train/RTE_slice:common_negation:ind/loss=7.63e-03, RTE_valid/RTE_slice:common_negation:ind/accuracy=9.75e-01] RTE_slice:common_negation:pred:[RTE_train/RTE_slice:common_negation:pred/loss=2.62e-03, RTE_valid/RTE_slice:common_negation:pred/accuracy=8.48e-01] RTE_slice:has_numerical_date:ind:[RTE_train/RTE_slice:has_numerical_date:ind/loss=1.34e-02, RTE_valid/RTE_slice:has_numerical_date:ind/accuracy=9.82e-01] RTE_slice:has_numerical_date:pred:[RTE_train/RTE_slice:has_numerical_date:pred/loss=2.20e-03, RTE_valid/RTE_slice:has_numerical_date:pred/accuracy=1.00e

[49.54 epo]: RTE:[RTE_train/RTE_gold/loss=6.92e-02, RTE_valid/RTE_gold/accuracy=7.55e-01] RTE_slice:has_preposition_s2:ind:[RTE_train/RTE_slice:has_preposition_s2:ind/loss=6.81e-03, RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy=9.82e-01] RTE_slice:has_preposition_s2:pred:[RTE_train/RTE_slice:has_preposition_s2:pred/loss=7.31e-03, RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy=7.55e-01] RTE_slice:common_negation:ind:[RTE_train/RTE_slice:common_negation:ind/loss=9.72e-03, RTE_valid/RTE_slice:common_negation:ind/accuracy=9.68e-01] RTE_slice:common_negation:pred:[RTE_train/RTE_slice:common_negation:pred/loss=3.28e-03, RTE_valid/RTE_slice:common_negation:pred/accuracy=7.88e-01] RTE_slice:has_numerical_date:ind:[RTE_train/RTE_slice:has_numerical_date:ind/loss=1.38e-02, RTE_valid/RTE_slice:has_numerical_date:ind/accuracy=9.82e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=9.06e-03, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:more_peop


[50.01 epo]: RTE:[RTE_train/RTE_gold/loss=8.65e-02, RTE_valid/RTE_gold/accuracy=7.33e-01] RTE_slice:has_preposition_s2:ind:[RTE_train/RTE_slice:has_preposition_s2:ind/loss=8.48e-03, RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy=9.86e-01] RTE_slice:has_preposition_s2:pred:[RTE_train/RTE_slice:has_preposition_s2:pred/loss=5.10e-03, RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy=7.35e-01] RTE_slice:common_negation:ind:[RTE_train/RTE_slice:common_negation:ind/loss=7.82e-03, RTE_valid/RTE_slice:common_negation:ind/accuracy=9.64e-01] RTE_slice:common_negation:pred:[RTE_train/RTE_slice:common_negation:pred/loss=2.42e-02, RTE_valid/RTE_slice:common_negation:pred/accuracy=7.27e-01] RTE_slice:has_numerical_date:ind:[RTE_train/RTE_slice:has_numerical_date:ind/loss=1.35e-02, RTE_valid/RTE_slice:has_numerical_date:ind/accuracy=9.82e-01] RTE_slice:has_numerical_date:pred:[RTE_train/RTE_slice:has_numerical_date:pred/loss=1.22e-03, RTE_valid/RTE_slice:has_numerical_date:pred/accuracy=1.00

/dfs/scratch0/vschen/metal-mmtl/metal/mmtl/trainer.py:915: UserWarning: MeTaL does not support calculating loss on the test set.
  warnings.warn(msg)


{'RTE/RTE_test/RTE_gold/accuracy': 0.0,
 'RTE_slice:common_negation:ind/RTE_test/RTE_slice:common_negation:ind/accuracy': 0.8853333333333333,
 'RTE_slice:common_negation:pred/RTE_test/RTE_slice:common_negation:pred/accuracy': 0.0,
 'RTE_slice:dash_semicolon:ind/RTE_test/RTE_slice:dash_semicolon:ind/accuracy': 0.6886666666666666,
 'RTE_slice:dash_semicolon:pred/RTE_test/RTE_slice:dash_semicolon:pred/accuracy': 0.0,
 'RTE_slice:has_numerical_date:ind/RTE_test/RTE_slice:has_numerical_date:ind/accuracy': 0.971,
 'RTE_slice:has_numerical_date:pred/RTE_test/RTE_slice:has_numerical_date:pred/accuracy': 0.0,
 'RTE_slice:has_preposition_s2:ind/RTE_test/RTE_slice:has_preposition_s2:ind/accuracy': 0.846,
 'RTE_slice:has_preposition_s2:pred/RTE_test/RTE_slice:has_preposition_s2:pred/accuracy': 0.0,
 'RTE_slice:more_people:ind/RTE_test/RTE_slice:more_people:ind/accuracy': 0.9776666666666667,
 'RTE_slice:more_people:pred/RTE_test/RTE_slice:more_people:pred/accuracy': 0.0}
Cleaning checkpoints
Writin

#### Did we improve?

In [148]:
%%time
slice_model.score(payloads_slice[1])

CPU times: user 3.7 s, sys: 776 ms, total: 4.47 s
Wall time: 4.4 s


{'RTE/RTE_valid/RTE_gold/accuracy': 0.8231046931407943,
 'RTE_slice:has_preposition_s2:ind/RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy': 0.8411552346570397,
 'RTE_slice:has_preposition_s2:pred/RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy': 0.8284313725490197,
 'RTE_slice:common_negation:ind/RTE_valid/RTE_slice:common_negation:ind/accuracy': 0.8953068592057761,
 'RTE_slice:common_negation:pred/RTE_valid/RTE_slice:common_negation:pred/accuracy': 0.8787878787878788,
 'RTE_slice:has_numerical_date:ind/RTE_valid/RTE_slice:has_numerical_date:ind/accuracy': 0.9819494584837545,
 'RTE_slice:has_numerical_date:pred/RTE_valid/RTE_slice:has_numerical_date:pred/accuracy': 1.0,
 'RTE_slice:more_people:ind/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.9566787003610109,
 'RTE_slice:more_people:pred/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.6666666666666666,
 'RTE_slice:dash_semicolon:ind/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.6101083032490975,
 'RTE_slice:dash_s

In [149]:
slice_model.score(eval_payload)

{'RTE/RTE_valid/RTE_gold/accuracy': 0.8231046931407943,
 'RTE/RTE_valid/RTE_slice:has_preposition_s2:ind/accuracy': 0.49097472924187724,
 'RTE/RTE_valid/RTE_slice:has_preposition_s2:pred/accuracy': 0.8235294117647058,
 'RTE/RTE_valid/RTE_slice:common_negation:ind/accuracy': 0.44404332129963897,
 'RTE/RTE_valid/RTE_slice:common_negation:pred/accuracy': 0.8787878787878788,
 'RTE/RTE_valid/RTE_slice:has_numerical_date:ind/accuracy': 0.4657039711191336,
 'RTE/RTE_valid/RTE_slice:has_numerical_date:pred/accuracy': 0.8,
 'RTE/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.44765342960288806,
 'RTE/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.6666666666666666,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.4981949458483754,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.7894736842105263}